# **Tugas Praktikum**


1. Buatlah model SVM dengan menggunakan data voice.csv dengan ketentuan,

  - Split data dengan rasio 70:30 dan 80:20 untuk setiap model yang akan dibangun.

    - Gunakan model dengan kernel linier.

    - Gunakan model dengan kernel polynomial.

    - Gunakan model dengan kernel RBF.

  - Tabulasikan performansi setiap split dan kernel berdasarkan metrik akurasi.

2. Gunakan data pada praktikum 5 untuk membuat model klasifikasi siang dan malam menggunakan SVM dengan kernel RBF menggunakan fitur histrogram. Gunakan rasio 80:20. Anda dapat bereksperimen dengan hyperparameter tunning dari kernel RBF. Catat performansi akurasinya!

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/voice.csv')
print(df.head())

   meanfreq        sd    median       Q25       Q75       IQR       skew  \
0  0.059781  0.064241  0.032027  0.015071  0.090193  0.075122  12.863462   
1  0.066009  0.067310  0.040229  0.019414  0.092666  0.073252  22.423285   
2  0.077316  0.083829  0.036718  0.008701  0.131908  0.123207  30.757155   
3  0.151228  0.072111  0.158011  0.096582  0.207955  0.111374   1.232831   
4  0.135120  0.079146  0.124656  0.078720  0.206045  0.127325   1.101174   

          kurt    sp.ent       sfm  ...  centroid   meanfun    minfun  \
0   274.402906  0.893369  0.491918  ...  0.059781  0.084279  0.015702   
1   634.613855  0.892193  0.513724  ...  0.066009  0.107937  0.015826   
2  1024.927705  0.846389  0.478905  ...  0.077316  0.098706  0.015656   
3     4.177296  0.963322  0.727232  ...  0.151228  0.088965  0.017798   
4     4.333713  0.971955  0.783568  ...  0.135120  0.106398  0.016931   

     maxfun   meandom    mindom    maxdom   dfrange   modindx  label  
0  0.275862  0.007812  0.007812  

In [ ]:
X_voice = df.drop('label', axis=1)
y_voice = df['label']

print("Features (X_voice) shape:", X_voice.shape)
print("Target (y_voice) shape:", y_voice.shape)

Features (X_voice) shape: (3168, 20)
Target (y_voice) shape: (3168,)


In [ ]:
from sklearn.preprocessing import LabelEncoder, StandardScaler

# Encode the target variable 'y_voice'
le = LabelEncoder()
y_voice_encoded = le.fit_transform(y_voice)

# Scale the features in X_voice
scaler = StandardScaler()
X_voice_scaled = scaler.fit_transform(X_voice)

print("Encoded target (y_voice_encoded) unique values:", np.unique(y_voice_encoded))
print("Scaled features (X_voice_scaled) shape:", X_voice_scaled.shape)

Encoded target (y_voice_encoded) unique values: [0 1]
Scaled features (X_voice_scaled) shape: (3168, 20)


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
import pandas as pd

# Dictionary to store results
results = {}

print("Initialized results dictionary and imported necessary libraries.")

Initialized results dictionary and imported necessary libraries.


In [ ]:
split_ratios = {'70:30': 0.3, '80:20': 0.2}
kernels = ['linear', 'poly', 'rbf']

for name, test_size in split_ratios.items():
    # Split data
    X_train, X_test, y_train, y_test = train_test_split(X_voice_scaled, y_voice_encoded, test_size=test_size, random_state=42)

    for kernel in kernels:
        # Train SVM model
        model = SVC(kernel=kernel, random_state=42)
        model.fit(X_train, y_train)

        # Make predictions
        y_pred = model.predict(X_test)

        # Calculate accuracy
        accuracy = accuracy_score(y_test, y_pred)

        # Store results
        if name not in results:
            results[name] = {}
        results[name][kernel] = accuracy

# Convert results to a pandas DataFrame for tabular display
results_df = pd.DataFrame(results)

print("SVM Model Training and Evaluation Complete. Results:")
print(results_df.to_markdown(numalign="left", stralign="left"))

SVM Model Training and Evaluation Complete. Results:
|        | 70:30    | 80:20    |
|:-------|:---------|:---------|
| linear | 0.970557 | 0.976341 |
| poly   | 0.956887 | 0.968454 |
| rbf    | 0.981073 | 0.98265  |


In [ ]:
def extract_histogram_features(image):
    # Convert the image from RGB to HSV color space
    hsv = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)

    # Calculate histograms for H, S, and V channels
    # H (Hue): 0-179, S (Saturation): 0-255, V (Value): 0-255
    hist_h = cv2.calcHist([hsv], [0], None, [180], [0, 180])
    hist_s = cv2.calcHist([hsv], [1], None, [256], [0, 256])
    hist_v = cv2.calcHist([hsv], [2], None, [256], [0, 256])

    # Normalize histograms
    cv2.normalize(hist_h, hist_h)
    cv2.normalize(hist_s, hist_s)
    cv2.normalize(hist_v, hist_v)

    # Concatenate the histograms into a single feature vector
    features = np.concatenate((hist_h.flatten(), hist_s.flatten(), hist_v.flatten()))

    return features

print("Histogram feature extraction function defined.")

Histogram feature extraction function defined.


In [ ]:
X_img_features = []
y_img_labels = []

for img_data, label in train_std_img_list:
    features = extract_histogram_features(img_data)
    X_img_features.append(features)
    y_img_labels.append(label)

X_img_features = np.array(X_img_features)
y_img_labels = np.array(y_img_labels)

# Split data with 80:20 ratio
X_train_img, X_test_img, y_train_img, y_test_img = train_test_split(X_img_features, y_img_labels, test_size=0.2, random_state=42)

print("Image features extracted and data split into training and testing sets.")
print(f"X_train_img shape: {X_train_img.shape}")
print(f"y_train_img shape: {y_train_img.shape}")
print(f"X_test_img shape: {X_test_img.shape}")
print(f"y_test_img shape: {y_test_img.shape}")

Image features extracted and data split into training and testing sets.
X_train_img shape: (192, 692)
y_train_img shape: (192,)
X_test_img shape: (48, 692)
y_test_img shape: (48,)


In [ ]:
from sklearn.model_selection import GridSearchCV

# Define the parameter grid for GridSearchCV
param_grid_img = {
    'C': [0.1, 1, 10, 100],
    'gamma': [0.0001, 0.001, 0.01, 0.1]
}

# Initialize the SVC model with RBF kernel
svc_model = SVC(kernel='rbf', random_state=42)

# Initialize GridSearchCV
grid_search_img = GridSearchCV(svc_model, param_grid_img, cv=5, scoring='accuracy', n_jobs=-1)

# Fit GridSearchCV to the training data
%time grid_search_img.fit(X_train_img, y_train_img)

print("Hyperparameter tuning complete.")
print(f"Best parameters: {grid_search_img.best_params_}")
print(f"Best cross-validation score: {grid_search_img.best_score_:.4f}")

CPU times: user 160 ms, sys: 19 ms, total: 179 ms
Wall time: 5.73 s
Hyperparameter tuning complete.
Best parameters: {'C': 10, 'gamma': 0.1}
Best cross-validation score: 1.0000


In [ ]:
best_svc_model = grid_search_img.best_estimator_
y_pred_img = best_svc_model.predict(X_test_img)

final_accuracy_img = accuracy_score(y_test_img, y_pred_img)

print("Final Image Classification Model Performance:")
print(f"Best Hyperparameters: {grid_search_img.best_params_}")
print(f"Test Accuracy: {final_accuracy_img:.4f}")

Final Image Classification Model Performance:
Best Hyperparameters: {'C': 10, 'gamma': 0.1}
Test Accuracy: 1.0000
